<a href="https://colab.research.google.com/github/ikyath/RNN_Introduction_Fundamentals/blob/master/RNN_Theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

intro
